In [1]:
import time
import pandas as pd
import numpy as np
import plotly.express as px
from xgboost import XGBRegressor

In [4]:
df_train = pd.read_parquet("df_train_pca.parquet").rename(columns={"forcast_hours":"hours_after", "Temperature_x":"temp_hornsea", "Temperature_y":"temp_solar"}).dropna()
df_test = pd.read_parquet("df_test_pca.parquet").dropna()
df_train["weekday"] = df_train.valid_time.dt.weekday
df_test["weekday"] = df_test.valid_time.dt.weekday

In [5]:
# label = "DA_Price"
label = "SS_Price"
columns = ["year", "month", "day", "hour", "season_autumn", "season_spring", "season_summer", "season_winter", "time_of_day_afternoon", "time_of_day_morning", "time_of_day_night", "weekday"]#, "MIP"

In [7]:
df_train[columns]

,year,month,day,hour,season_autumn,season_spring,season_summer,season_winter,time_of_day_afternoon,time_of_day_morning,time_of_day_night,weekday
1,2020,9,20,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,6
2,2020,9,20,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,6
3,2020,9,20,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,6
4,2020,9,20,2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,6
5,2020,9,20,2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...
1273718,2024,5,19,21,0.0,1.0,0.0,0.0,0.0,0.0,1.0,6
1273719,2024,5,19,22,0.0,1.0,0.0,0.0,0.0,0.0,1.0,6
1273720,2024,5,19,22,0.0,1.0,0.0,0.0,0.0,0.0,1.0,6
1273721,2024,5,19,23,0.0,1.0,0.0,0.0,0.0,0.0,1.0,6


### ARIMA

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
prices = df_train.loc[(df_train["hours_after"] <= 5.5) & (df_train["valid_time"] <= "2023-09-01")].sort_values("valid_time").reset_index(drop=True).DA_Price.values
dates = df_train.loc[(df_train["hours_after"] <= 5.5) & (df_train["valid_time"] <= "2023-09-01")].set_index("valid_time").resample("30T").interpolate("linear",limit=5).reset_index().valid_time.values

C:\Users\manu\AppData\Local\Temp\ipykernel_8468\320526429.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  dates = df_train.loc[(df_train["hours_after"] <= 5.5) & (df_train["valid_time"] <= "2023-09-01")].set_index("valid_time").resample("30T").interpolate("linear",limit=5).reset_index().valid_time.values
C:\Users\manu\AppData\Local\Temp\ipykernel_8468\320526429.py:2: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  dates = df_train.loc[(df_train["hours_after"] <= 5.5) & (df_train["valid_time"] <= "2023-09-01")].set_index("valid_time").resample("30T").interpolate("linear",limit=5).reset_index().valid_time.values


In [ ]:
model = ARIMA(endog=prices, dates=dates)
model = model.fit()

c:\Users\manu\py_venvs\DoPro2\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 30min will be used.
  self._init_dates(dates, freq)
c:\Users\manu\py_venvs\DoPro2\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 30min will be used.
  self._init_dates(dates, freq)


In [ ]:
model.predict(pd.date_range(start=df_test.valid_time.min(), end=df_test.valid_time.max())[0])

KeyError: 'The `start` argument could not be matched to a location related to the index of the data.'

### XGBoost

In [8]:
x = df_train[columns].to_numpy()
y = df_train[label].to_numpy()

x_test = df_test[columns].to_numpy()
y_test = df_test[label].to_numpy()

In [37]:
model = XGBRegressor(gamma=20, eta=0.001, max_depth=10, n_estimators=2000)#device="cuda",
model.fit(x, y)

preds = model.predict(x)
preds_test = model.predict(x_test)

print(f"Trainingsfehler = {abs(preds - y).mean().round(2)}")
print(f"Testfehler = {abs(preds_test - y_test).mean().round(2)}")

Trainingsfehler = 41.03
Testfehler = 49.21


In [38]:
model.feature_importances_

array([0.74969876, 0.02608879, 0.00797888, 0.00437421, 0.00119227,
       0.00733076, 0.13830514, 0.00737848, 0.01759401, 0.00215115,
       0.03149604, 0.00641154], dtype=float32)

In [39]:
model.save_model("ss_price_ohne_MIP.json")

In [10]:
nestimators = [1000]
depths = [4, 8]
etas = [0.005, 0.01, 0.05, 0.1, 0.3]
gammas = [0]
results = []

for ne in nestimators:
  print(f"Nest: {ne}")
  for d in depths:
    print(f"Depth: {d}")
    for e in etas:
      for gamma in gammas:
        model = XGBRegressor(device="cuda", gamma=gamma, eta=e, max_depth=d, n_estimators=ne)
        start = time.time()
        model.fit(x, y)
        end = time.time()
        preds = model.predict(x)
        preds_test = model.predict(x_test)
        results.append((ne, d, e, gamma, end-start, abs(preds - y).mean().round(2), abs(preds_test - y_test).mean().round(2)))

Nest: 1000
Depth: 4


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [12:37:00] WARNING: /workspace/src/context.cc:43: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [12:37:00] WARNING: /workspace/src/context.cc:196: XGBoost is not compiled with CUDA support.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [12:38:19] WARNING: /workspace/src/context.cc:43: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [12:38:19] WARNING: /workspace/src/context.cc:196: XGBoost is not compiled with CUDA support.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [12:39:31] WARNING: /workspace/src/context.cc:43: No visible GPU is found, setting device to CPU.
  warnin

Depth: 8


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [12:43:15] WARNING: /workspace/src/context.cc:43: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [12:43:15] WARNING: /workspace/src/context.cc:196: XGBoost is not compiled with CUDA support.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [12:45:46] WARNING: /workspace/src/context.cc:43: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [12:45:46] WARNING: /workspace/src/context.cc:196: XGBoost is not compiled with CUDA support.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [12:48:14] WARNING: /workspace/src/context.cc:43: No visible GPU is found, setting device to CPU.
  warnin

In [11]:
pd.DataFrame(data=np.array(results).round(2), columns=["nestimators", "depth", "eta", "gamma", "Zeit", "train_score", "test_score"]).to_excel("scores.xlsx")

In [ ]:
df_show = pd.DataFrame(data={"date":df_test.valid_time, "y": y_test, "pred":preds})

In [ ]:
px.line(df_show.sort_values("date"), x="date", y=["y", "pred"])

Output hidden; open in https://colab.research.google.com to view.